### Description

This notebook is intended to demo how the Supernova SDK handles the In-Band Interrupts notification sent by the Supernova. The notebooks uses the TDK ICM42605 as the I3C Target, and runs a sequence of I3C Private transfer to configure the target to request In-Band Interrupts. 

In [1]:
# Import Python package
from BinhoSupernova import getConnectedSupernovaDevicesList
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *

#### 1. List all the Supernova devices conneted to the PC host

In [2]:
getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&f321146&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '59314AC593A1C05C8163B6759994476',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': -1}]

#### 2. Connect to Supernova device

In [3]:
# Create device instance
supernova = Supernova()
supernova.open()

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

In [4]:
import json

# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:
    
    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")

        # If the command is GET TARGET DEVICE TABLE, convert to json.
        if supernova_message["name"] == "I3C GET TARGET DEVICE TABLE" or supernova_message["name"] == "I3C TRANSFER":
            responseJson = json.dumps(supernova_message, indent=4)
            print(responseJson)

        else:
            print(supernova_message)
    
    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")

        print(system_message)

In [5]:
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 3. Define a function to generate transfers ID

In [6]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

In [7]:
supernova.i3cControllerInit(getId())

{'module': 1,
 'opcode': 0,
 'message': 'INITIALIZE I3C CONTROLLER requests success'}

>> New message from SUPERNOVA:
{'id': 1, 'command': 1, 'name': 'I3C CONTROLLER INIT', 'result': 'I3C_CONTROLLER_INIT_SUCCESS'}


In [8]:
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

>> New message from SUPERNOVA:
{'id': 2, 'command': 96, 'name': 'GET USB STRING', 'length': 20, 'message': 'FW-1.3.0-187-fc74615'}


### I3C Protocol

In [9]:
request_result = supernova.setI3cBusVoltage(getId(), 3300)

>> New message from SUPERNOVA:
{'id': 3, 'command': 97, 'name': 'SET I3C BUS VOLTAGE', 'result': 0}


### 1. I3C bus initialization

There are two options:
- Initialize the bus with an empty target devices table and scan the I3C bus.
- Add targets to the target device table before initializing the bus.

In [10]:
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 4,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


Add targets to the target devices table using a Python dictionary

In [11]:
ICM_STATIC_ADDRESS = 0x69
ICM_DYNAMIC_ADDRESS = 0x08

features =  TargetInterruptRequest.ACCEPT_INTERRUPT_REQUEST.value |  \
            ControllerRoleRequest.REJECT_CRR.value |   \
            IBiTimestamp.DISABLE_IBIT.value |   \
            AssignmentFromStaticAddress.I3C_TARGET_SUPPORTS_SETDASA.value |   \
            DAaWithENTDAA.ENABLE_ENTDAA.value |   \
            TargetType.I3C_DEVICE.value |   \
            PendingReadCapability.SUPPORT_IBI_READ_CAPABILITY.value |   \
            ValidPID.HAS_VALID_PID.value

table = {0: {'staticAddress': ICM_STATIC_ADDRESS,
        'dynamicAddress': ICM_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'maxIbiPayloadLength': 9,
        'i3cFeatures': features}}

request_result = supernova.i3cInitBus(getId(), table)

>> New message from SUPERNOVA:
{'id': 5, 'command': 2, 'name': 'I3C INIT BUS', 'result': 'I3C_BUS_INIT_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [12]:
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 6,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 105,
            "dynamicAddress": 8,
            "bcr": {
                "value": [
                    "0b00000000",
                    0,
                    "0x00"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Does not support optional advanced capabilities.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "No data bytes follow the accepted IBI.",
                    "ibiRequestCapable": "Not capable.",
                    "maxDataSpeedLimitation": "No Limitation."
          

### 2. I3C Transfers

#### 2.1 Update PID, BCR and DCR

In [13]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETPID(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 7,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 6,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "pid": [
        "0x04",
        "0x6a",
        "0x00",
        "0x00",
        "0x00",
        "0x00"
    ]
}


In [14]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETBCR(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ)

>> New message from SUPERNOVA:
{
    "id": 8,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "bcr": {
        "value": [
            "0b00100111",
            39,
            "0x27"
        ],
        "description": {
            "deviceRole": "I3C Target.",
            "advancedCapabilities": "Supports optional advanced capabilities. Use GETCAPS CCC to deremine which ones.",
            "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
            "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
            "ibiPayload": "One data byte (MDB) shall follow the accepted IBI, and additional data bytes may follow.",
            "ibiRequestCap

In [15]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETDCR(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ)

>> New message from SUPERNOVA:
{
    "id": 9,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "dcr": "0xa0"
}


#### 2.2 Set MWL and MRL

In [16]:
TARGET_ADDR = 0x08
MAXIMUM_READ_LEN = 512

request_result = supernova.i3cDirectSETMRL(getId(), 
                            TARGET_ADDR, 
                            I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ,
                            I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                            MAXIMUM_READ_LEN)

>> New message from SUPERNOVA:
{
    "id": 10,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


In [17]:
MAXIMUM_WRITE_LEN = 256

request_result = supernova.i3cBroadcastSETMWL(getId(),
                                                I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ,
                                                I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                                MAXIMUM_WRITE_LEN)

>> New message from SUPERNOVA:
{
    "id": 11,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 3,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0,
        0
    ]
}


#### 2.3 Disable In-Band interrupts

In [18]:
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.REGULAR_IBI, ICM_DYNAMIC_ADDRESS)

>> New message from SUPERNOVA:
{'id': 12, 'command': 7, 'name': 'I3C CLEAR FEATURE', 'result': 'I3C_CLEAR_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


#### 2.4 Configure sensor

1. Write ``REG_BANK_SEL``

Change register bank to bank 0.

In [19]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x00

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [REG_BANK_SEL], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 13,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


2. Write ``PWR_MGMT0`` register

Turn off Accelerometer to modify registers value.

In [20]:
# PWR_MGMT0 register address
PWR_MGMT0   = 0x4E
VALUE       = 0x20

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [PWR_MGMT0], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 14,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


3. Write ``DRIVE_CONFIG`` register

Set ``I2C_SLEW_RATE`` and ``SPI_SLEW_RATE``

In [21]:
# DRIVE_CONFIG register address
DRIVE_CONFIG    = 0x13
VALUE           = 0x05

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [DRIVE_CONFIG], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 15,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


4. Write ``FIFO_CONFIG`` register

Set Fifo mode: Stream-to-FIFO Mode

In [22]:
# FIFO_CONFIG register address
FIFO_CONFIG = 0x16
VALUE       = 0x40

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [FIFO_CONFIG], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 16,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


5. Write ``FIFO_CONFIG1`` register

- FIFO read can be partial, and resume from last read point
- Trigger FIFO watermark interrupt on every ODR (DMA write) if FIFO_COUNT ≥ FIFO_WM_TH
- Enable accelerometer packets to go to FIFO

In [23]:
# FIFO_CONFIG1 register address
FIFO_CONFIG1    = 0x5F
VALUE           = 0x61

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [FIFO_CONFIG1], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 17,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


In [24]:
# FIFO_CONFIG2 register address
FIFO_CONFIG2   = 0x60
VALUE          = [0x0F, 0x00]

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [FIFO_CONFIG2], 
                                    VALUE)

>> New message from SUPERNOVA:
{
    "id": 18,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


7. Write ``ACCEL_CONFIG0``

Set Accelerometer ODR selection for UI interface output to 1.5625Hz (LP mode).

In [25]:
# ACCEL_CONFIG0 register address
ACCEL_CONFIG0   = 0x50
#VALUE           = 0x0A # 25Hz (LP or LN mode)
VALUE           = 0x0E # 1.5625Hz (LP mode)

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [ACCEL_CONFIG0], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 19,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


8. Write ``REG_BANK_SEL``

Change register bank to bank 1.

In [26]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x01

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [REG_BANK_SEL], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 20,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


9. Write ``SENSOR_CONFIG0``

Enable accelerometer X, Y, and Z data.

In [27]:
# SENSOR_CONFIG0 register address
SENSOR_CONFIG0  = 0x03
VALUE           = 0x38

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [SENSOR_CONFIG0], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 21,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


10. Write ``INTF_CONFIG4``

Set I3C_BUS_MODE

In [28]:
# INTF_CONFIG4 register address
INTF_CONFIG4    = 0x7A
VALUE           = 0x02

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [INTF_CONFIG4], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 22,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


11. Write ``INTF_CONFIG6``

Enable I3C In-Band Interrupt

In [29]:
# INTF_CONFIG6 register address
INTF_CONFIG6    = 0x7C
VALUE           = 0x1F

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [INTF_CONFIG6], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 23,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


12. Write ``REG_BANK_SEL``

Change register bank to bank 4.

In [30]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x04

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [REG_BANK_SEL], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 24,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


13. Write ``INT_SOURCE8``

Route FIFO threshold interrupt to I3C In-Band Interrupt

In [31]:
# INT_SOURCE8 register address
INT_SOURCE8 = 0x4F
VALUE       = 0x04

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [INT_SOURCE8], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 25,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


14. Write ``REG_BANK_SEL``

Change register bank to bank 0.

In [32]:
# REG_BANK_SEL register address
REG_BANK_SEL    = 0x76
VALUE           = 0x00

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [REG_BANK_SEL], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 26,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


15. Write ``PWR_MGMT0`` register

Enable Temperature sensor and Accelerometer in Low Power mode

In [33]:
# PWR_MGMT0 register address
PWR_MGMT0   = 0x4E
#VALUE       = 0x03 # 11: Places accelerometer in Low Noise (LN) Mode
VALUE       = 0x02 # 10: Places accelerometer in Low Power (LP) Mode

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    ICM_DYNAMIC_ADDRESS,
                                    TransferMode.I3C_SDR,
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ, 
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_4_17_MHZ,
                                    [PWR_MGMT0], 
                                    [VALUE])

>> New message from SUPERNOVA:
{
    "id": 27,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


#### 2.4 Enable In-Band interrupts

In [34]:
request_result = supernova.i3cSetFeature(getId(), I3cSetFeatureSelector.REGULAR_IBI, ICM_DYNAMIC_ADDRESS)

>> New message from SUPERNOVA:
{'id': 28, 'command': 8, 'name': 'I3C SET FEATURE', 'result': 'I3C_SET_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}
>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 1, 'status': 0}, 'MDB': {'value': 2, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 1, 'status': 0}, 'MDB': {'value': 2, 'description

In [35]:
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.REGULAR_IBI, ICM_DYNAMIC_ADDRESS)

>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 1, 'status': 0}, 'MDB': {'value': 2, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [36]:
request_result = supernova.i3cSetFeature(getId(), I3cSetFeatureSelector.REGULAR_IBI, 0x7E)

>> New message from SUPERNOVA:
{'id': 29, 'command': 8, 'name': 'I3C SET FEATURE', 'result': 'I3C_CLEAR_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}
>> New message from SUPERNOVA:
{'id': 30, 'command': 8, 'name': 'I3C SET FEATURE', 'result': 'I3C_SET_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}
>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 1, 'status': 0}, 'MDB': {'value': 2, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATI

In [37]:
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.REGULAR_IBI, 0X7E)

>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 1, 'status': 0}, 'MDB': {'value': 2, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
